In [1]:
import os
current_dir = %pwd
project_dir = os.path.dirname(current_dir)
%cd $project_dir

c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier


c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

import pandas as pd
from joblib import dump, load
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/processed/words_structure.csv')
seed = 42


C:\Users\ayoub\AppData\Local\Temp\ipykernel_85976\2076755922.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df['words'] = df['words'].fillna('')

target = df['category']
features = df.drop('category', axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=seed)
le = load("models/target_LabelEncoder.joblib")
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

In [5]:
X_train_structure = X_train.drop('words', axis=1)
X_test_structure = X_test.drop('words', axis=1)

scaler = load('models/StandardScaler.joblib')
X_train_scaled = scaler.transform(X_train_structure)
X_test_scaled = scaler.transform(X_test_structure)

I- logistic regression model

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the Logistic Regression model
model = LogisticRegression()

# Define the different parameters to test
param_grid = {                                                 
    'penalty': ['l2'],  # Only 'l2' penalty is compatible with 'lbfgs' solver
    'C': [0.01, 0.1, 1, 10, 100, 200],                                  
    'solver': ['lbfgs'],  # Use only 'lbfgs' solver
    'max_iter': [100, 200, 300, 400]  # Increase max_iter or use other solvers if convergence issue persists
}

# Create the GridSearchCV object with verbose=3
grid_search_lr = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_lr.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_lr = grid_search_lr.best_params_
best_score_lr = grid_search_lr.best_score_

# Evaluate the model on the test data
test_score_lr = grid_search_lr.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_lr)
print("Best score on training data:", best_score_lr)
print("Score on test data:", test_score_lr)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters found: {'C': 200, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score on training data: 0.8162489392583387
Score on test data: 0.820675105485232


c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Save logistic regression model 
best_model_logistic_regression = grid_search_lr.best_estimator_
dump(best_model_logistic_regression, 'models/logistic_regression_structure.joblib')

II -   lightgbm.LGBMClassifier model 
 

In [13]:
# Approximatively 39 min

from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

# Define the LGBMClassifier model
model = LGBMClassifier()

# Define the different parameters to test
param_grid = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9, 1.0],
    }

# Create the GridSearchCV object with verbose=3
grid_search_lgm = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_lgm.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_lgm = grid_search_lgm.best_params_
best_score_lgm = grid_search_lgm.best_score_

# Evaluate the model on the test data
test_score_lgm = grid_search_lgm.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_lgm)
print("Best score on training data:", best_score_lgm)
print("Score on test data:", test_score_lgm)


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001024 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6370
[LightGBM] [Info] Number of data points in the train set: 5687, number of used features: 26
[LightGBM] [Info] Start training from score -1.751268
[LightGBM] [Info] Start training from score -1.714466
[LightGBM] [Info] Start training from score -1.406723
[LightGBM] [Info] Start training from score -3.648726
[LightGBM] [Info] Start training from score -1.608032
[LightGBM] [Info] Start training from score -1.742191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

In [10]:
# Save LightGBM model 
best_LGBM = grid_search_lgm.best_estimator_
dump(best_LGBM, 'models/LGBM_classifier_structure.joblib')

['models/LGBM_classifier_structure.joblib']

III - RANDOM FOREST MODEL 

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
0
# Define the RandomForestClassifier model
model = RandomForestClassifier()

# Define the different parameters to test
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    
}

# Create the GridSearchCV for random forest
grid_search_rf = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_rf.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_rf = grid_search_rf.best_params_
best_score_rf = grid_search_rf.best_score_

# Evaluate the model on the test data
test_score_rf = grid_search_rf.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_rf)
print("Best score on training data:", best_score_rf)
print("Score on test data:", test_score_rf)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best score on training data: 0.8354146282651136
Score on test data: 0.820675105485232


In [12]:
# Save random forest model 
best_rf = grid_search_rf.best_estimator_
dump(best_rf, 'models/random_forest_structure.joblib')

['models/random_forest_structure.joblib']

IV - XGBClassifier model

In [13]:
# Approximatively 38 min
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the XGBClassifier model
model = XGBClassifier()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.2],
}

# Create the GridSearchCV object with verbose=3
grid_search_xgb = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_xgb.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_

# Evaluate the model on the test data
test_score_xgb = grid_search_xgb.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_xgb)
print("Best score on training data:", best_score_xgb)
print("Score on test data:", test_score_xgb)




Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best score on training data: 0.8347121042796
Score on test data: 0.8255977496483825


In [14]:
# Save xgb classifier model 
best_xgb = grid_search_xgb.best_estimator_
dump(best_xgb, 'models/xgb_classifier_structure.joblib')

['models/xgb_classifier_structure.joblib']

V- ExtraTrees Classifier model  

In [19]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

# Define the ExtraTreesClassifier model
model = ExtraTreesClassifier()

# Define the simplified parameter grid
param_grid =  {                                             
    'n_estimators': [50, 100, 200, 300],                                
    'criterion': ['gini', 'entropy'],                                   
    'max_depth': [None, 20, 30],                                        
    'min_samples_split': [2, 5, 10],                                    
    'min_samples_leaf': [1, 2, 4],                                      
    'max_features': ['sqrt', 'log2'],                           
    'bootstrap': [True],                                                
    'oob_score': [True, False],                                         
}

# Create the GridSearchCV object with verbose=3
grid_search_et = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_et.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_et = grid_search_et.best_params_
best_score_et = grid_search_et.best_score_

# Evaluate the model on the test data
test_score_et = grid_search_et.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_et)
print("Best score on training data:", best_score_et)
print("Score on test data:", test_score_et)


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier\.venv\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
2160 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1130 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ayoub\OneDrive\Bureau\Datascientest_project\doc-classifier\.venv\lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\ayoub\OneDrive\B

Best parameters found: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100, 'oob_score': True}
Best score on training data: 0.8320752821302321
Score on test data: 0.8178621659634318


Best parameters found: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100, 'oob_score': True}
Best score on training data: 0.8320752821302321
Score on test data: 0.8178621659634318

In [ ]:
# Save ExtraTreesClassifier model
best_et = grid_search_et.best_estimator_
dump(best_et, 'models/extra_trees_classifier_structure.joblib')

VI- NearestCentroid Classifier model 

In [17]:
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import GridSearchCV

# Define the NearestCentroid model
model = NearestCentroid()

# Define the simplified parameter grid
param_grid = {
    'metric' : ['euclidean', 'manhattan'] ,
    'shrink_threshold': [None, 0.01, 10, 50],
}

# Create the GridSearchCV object with verbose=3
grid_search_nc = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_nc.fit(X_train_scaled, y_train_encoded)

# Once fitting is done, you can access the best parameters and best score
best_params_nc = grid_search_nc.best_params_
best_score_nc = grid_search_nc.best_score_

# Evaluate the model on the test data
test_score_nc = grid_search_nc.score(X_test_scaled, y_test_encoded)

print("Best parameters found:", best_params_nc)
print("Best score on training data:", best_score_nc)
print("Score on test data:", test_score_nc)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters found: {'metric': 'manhattan', 'shrink_threshold': None}
Best score on training data: 0.7174264591090852
Score on test data: 0.7243319268635724


In [18]:
# Save NearestCentroid Classifier model 
best_nc = grid_search_nc.best_estimator_
dump(best_nc, 'models/NearestCentroid_structure.joblib')

['models/NearestCentroid_structure.joblib']